In [40]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from pydub import AudioSegment
from tqdm import tqdm
import soundfile as sf
from sklearn.preprocessing import StandardScaler
from scipy.io import wavfile
from scipy import signal
from pydub import AudioSegment
from pydub.utils import json
import os

In [41]:
# Charger les fichiers audio et calculer leurs longueurs
audio_files = []
for filename in os.listdir("data/audio_gasy"):
    audio_path = os.path.join("data/audio_gasy", filename)
    if audio_path.endswith(".wav"):
        audio_files.append(audio_path)

audio_lengths = [librosa.get_duration(filename=file) for file in tqdm(audio_files)]

# Déterminer la longueur cible (par exemple, la plus grande longueur)
target_length = np.mean(audio_lengths)

output_file = "data/mean_length.json"

# Ecriture du fichier json
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(target_length, f, indent=4, ensure_ascii=False)
    
print(target_length)

  0%|          | 0/1824 [00:00<?, ?it/s]C:\Users\randy bidy\AppData\Local\Temp\ipykernel_7456\3206494570.py:8: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  audio_lengths = [librosa.get_duration(filename=file) for file in tqdm(audio_files)]
100%|██████████| 1824/1824 [00:23<00:00, 76.54it/s]

5.988771792763157


# Élimination des silences 

In [42]:
def remove_silence(audio_path, output):
    # Charger le fichier audio
    signal, sr = librosa.load(audio_path, sr=None)

    # Détection des régions actives dans le signal
    non_silent_intervals = librosa.effects.split(signal, top_db=30)

    # Fusionner les intervalles non silencieux
    non_silent_signal = librosa.effects.remix(signal, non_silent_intervals)

    # Sauvegarder le signal audio sans les silences
    sf.write(output, non_silent_signal, sr)

# Normalisation du volume

In [43]:
def normalize_audio_volume(audio_path, output_path, target_dBFS=-20.0):
    # Chargement de l'enregistrement audio
    audio = AudioSegment.from_file(audio_path)

    # Calcul du facteur de normalisation pour atteindre le niveau cible
    current_dBFS = audio.dBFS
    normalization_factor = (target_dBFS - current_dBFS)

    # Normalisation du volume de l'audio
    normalized_audio = audio + normalization_factor

    # Export de l'audio normalisé
    normalized_audio.export(output_path, format="wav")

# Filtrage du bruit

In [44]:
def filtrage_du_bruit(audio_path, output, noise_threshold=-40.0):
    # Chargement de l'enregistrement audio
    audio = AudioSegment.from_file(audio_path)

    # Détection du bruit de fond
    background_noise = audio.dBFS

    # Filtrer le bruit de fond
    if background_noise > noise_threshold:
        audio = audio - noise_threshold
    else:
        audio = audio - background_noise

    # Export de l'audio filtré
    audio.export(output, format="wav")

# Segmentation de la parole

In [45]:
def segmentation_parole(audio_path, output_file, silence_threshold=-45):
    # Charger le fichier audio
    audio = AudioSegment.from_file(audio_path, format="wav")

    # Détection des silences
    non_silent_audio = audio.strip_silence(silence_thresh=silence_threshold)

    # Exporter le fichier audio sans les silences
    non_silent_audio.export(output_file, format="wav")

# Éliminé des artefacts

In [46]:
def remove_artifacts(audio_path, output_path):
    # Chargement de l'enregistrement audio
    audio = AudioSegment.from_file(audio_path)

    # Suppression d'artefacts basée sur la fréquence ou l'amplitude
    # Par exemple, supprimer les fréquences inférieures à 1000 Hz
    audio_filtered = audio.low_pass_filter(1000)

    # Export de l'audio filtré
    audio_filtered.export(output_path, format="wav")
# Exemple d'utilisation

# Préaccentuation

In [47]:
from scipy.io import wavfile
def preaccentuation(audio_file, output):
    # Charger le signal vocal (remplacer "audio.wav" par votre propre fichier audio)
    sample_rate, audio_data = wavfile.read(audio_file)

    # Paramètres de la préaccentuation
    alpha = 0.95  # Facteur de préaccentuation (typiquement entre 0.9 et 1)

    # Appliquer la préaccentuation
    preemphasis_audio = np.append(audio_data[0], audio_data[1:] - alpha * audio_data[:-1])

    # Enregistrer le signal filtré en tant que fichier WAV
    wavfile.write(output, sample_rate, np.int16(preemphasis_audio))

# Normalisation temporelle

In [48]:
def time_stretch_audio(input_file, output_file, target_duration):
    # Charger l'audio
    audio, sr = librosa.load(input_file)

    # Calculer la durée actuelle
    current_duration = len(audio)/44100

    # Calculer le facteur de normalisation
    speed_factor = current_duration / target_duration

    # Normaliser l'audio en modifiant la vitesse
    normalized_audio = librosa.effects.time_stretch(y=audio, rate=speed_factor)

    # Sauvegarder l'audio normalisé
    sf.write(output_file, normalized_audio, sr)

# Utilisation de la fonction


# Convertir en_format adapté

In [49]:
import torchaudio
from torchvision.transforms import Compose
from torchaudio.transforms import MelSpectrogram

def convert_to_spectrogram(audio_path):
    transform = Compose([
        torchaudio.transforms.Resample(orig_freq=44100, new_freq=16000),  # Échantillonnage à 16 kHz
        MelSpectrogram(n_fft=400, win_length=400, hop_length=160, n_mels=128)  # Créer un spectrogramme Mel
    ])
    waveform, sample_rate = torchaudio.load(audio_path)
    spectrogram = transform(waveform).unsqueeze(0)  # Ajouter une dimension de lot
    spectrogram = np.array(spectrogram)
    spectrogram = spectrogram.reshape(-1)
    
    return spectrogram, sample_rate

# Correction d'accent 

In [50]:
def correction_accent(audio_file):
    signal, sample_rate = librosa.load(audio_file, sr=None)

    # Calculer les coefficients cepstraux MFCC
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=13)

    # Normaliser les coefficients MFCC
    scaler = StandardScaler()
    return scaler.fit_transform(mfccs.T).T

# Pretraitement audio
**Note:** Ne pas oublier de creer manuellement le dossier temp a la racine du projet

In [51]:
audio_dir = "data/audio_gasy/"
labels_train = []
# Liste pour stocker les caractéristiques MFCC de chaque fichier audio
mfcc_features_list = []
# Chargement des labels
with open("data/audio_path.json", "r", encoding="utf-8") as f:
    transcriptions = json.load(f)

for transcription in tqdm(transcriptions):
# for filename in tqdm(os.listdir(audio_dir)):
    audio_path = os.path.join(audio_dir, filename)
    if audio_path.endswith(".wav"):
        # audio_path = audio_dir+transcription["audio_file"]
        remove_silence(audio_path, "temp/remove_silence.wav")
        normalize_audio_volume("temp/remove_silence.wav", "temp/normalize_audio_volume.wav")
        time_stretch_audio("temp/normalize_audio_volume.wav", "temp/time_stretch_audio.wav", target_length)
        filtrage_du_bruit("temp/time_stretch_audio.wav", "temp/filtrage_du_bruit.wav")
        segmentation_parole("temp/filtrage_du_bruit.wav", "temp/segmentation_parole.wav")
        remove_artifacts("temp/segmentation_parole.wav", "temp/remove_artifacts.wav")
        preaccentuation("temp/remove_artifacts.wav", "temp/preaccentuation.wav")
        mfcc_features_normalized = correction_accent("temp/preaccentuation.wav")
        mfcc_features_list.append(mfcc_features_normalized)
        labels_train.append(transcription["transcription"])

  5%|▌         | 155/3000 [02:35<32:56,  1.44it/s] 

In [ ]:
# Convertir la liste en un tableau numpy
X = np.array(mfcc_features_list)
print(X.shape)
X = X.reshape(X.shape[0], -1)
print(X.shape)
# Enregistrement des caractéristiques prétraitées dans un fichier (optionnel)
np.save("data/mfcc_feature.npy", X)

(100, 13, 516)
(100, 6708)


In [ ]:
from sklearn.preprocessing import LabelEncoder
import re
# Convertir les labels en un format numérique (par exemple, utiliser un encodage one-hot)
for i in range(len(labels_train)):
    # Conversion en minuscules
    labels_train[i] = labels_train[i].lower()
    # Suppression de la ponctuation
    labels_train[i] = re.sub(r'[^\w\s]', '', labels_train[i])

# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(labels_train)
# y = np.array(y)
# print(y)
vocab = []
word_to_index = {}
index_to_word = {}
for sentence in labels_train:
    tokens = sentence.split()
    for token in tokens:
        word_to_index[token] = len(word_to_index)-1
        index_to_word[len(index_to_word)] = token
        vocab.append(token)
            
print(vocab)
# Encodage des séquences
encoded_data = []
for sentence in vocab:
    tokens = sentence.split()
    encoded_sentence = [word_to_index[token] for token in tokens]
    encoded_data.append(encoded_sentence)
encoded_data = np.array(encoded_data).reshape(-1)
np.save("data/labels.npy", encoded_data)

['zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero', 'zero']


# Cleaning
> Suppression des fichiers temporaires cree durant la pretraitement de donnee

In [ ]:
import os

# Obtenir le chemin du répertoire temporaire
temp_dir = "temp/"

# Parcourir les fichiers du répertoire temporaire
for file in os.listdir(temp_dir):
    # Vérifier si c'est un fichier
    if os.path.isfile(os.path.join(temp_dir, file)):
        # Supprimer le fichier
        os.remove(os.path.join(temp_dir, file))
